In [13]:
pip install wordcloud

Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install advertools

   ---------------------------------------- 0.0/322.0 kB ? eta -:--:--
   ----- ---------------------------------- 41.0/322.0 kB 1.9 MB/s eta 0:00:01
   --------------------- ------------------ 174.1/322.0 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 322.0/322.0 kB 2.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/151.7 kB ? eta -:--:--
   ---------------------------------------- 151.7/151.7 kB 4.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'demoji'

In [28]:
pip install demoji


   ---------------------------------------- 0.0/42.9 kB ? eta -:--:--
   ------------------- -------------------- 20.5/42.9 kB 682.7 kB/s eta 0:00:01
   ---------------------------------------- 42.9/42.9 kB 526.0 kB/s eta 0:00:00


In [30]:
import demoji

# Read the text file
with open("datanew.txt", "r", encoding="utf-8") as f:
  text = f.read()

# Remove emojis (consider replacing with demoji.replace(text, repdict) for text descriptions)
text_without_emojis = demoji.removeAll(text)

# Optionally, write the modified text to a new file
with open("datanew.txt", "w", encoding="utf-8") as f:
  f.write(text_without_emojis)

print("Emojis removed from text file.")


AttributeError: module 'demoji' has no attribute 'removeAll'

In [33]:
import re

def remove_undesired_chars(text):
  """
  This function removes emojis and unknown symbols from a text string using regular expressions.

  Args:
      text (str): The text string from which to remove undesired characters.

  Returns:
      str: The text string with emojis and unknown symbols removed.
  """
  undesired_pattern = r"[^\x00-\x7F\sA-Za-z0-9]"  # Matches characters outside basic ASCII, space, letters, and numbers
  return re.sub(undesired_pattern, '', text)

# Read the text file
with open("data.txt", "r", encoding="utf-8") as f:
  text = f.read()

# Remove undesired characters
text_without_unknowns = remove_undesired_chars(text)

# Optionally, write the modified text to a new file
with open("data.txt", "w", encoding="utf-8") as f:
  f.write(text_without_unknowns)

print("Emojis and unknown symbols removed from file.")


Emojis and unknown symbols removed from file.


In [19]:
#!/usr/bin/env python
"""
Remove emoji from a text file and print it to stdout.
Usage
-----
    python remove-emoji.py input.txt > output.txt
"""
import re
import sys

# https://stackoverflow.com/a/49146722/330558
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

if __name__ == '__main__':
    text = open(sys.argv[1]).read()
    text = remove_emoji(text)
    print(text)

FileNotFoundError: [Errno 2] No such file or directory: '-f'

In [50]:
import warnings 
warnings.filterwarnings('ignore') 
import pandas as pd 
import re 
import seaborn as sns 
from sklearn.feature_extraction.text import TfidfVectorizer 
import matplotlib.pyplot as plt 
from wordcloud import WordCloud


In [ ]:
import nltk 
nltk.download('stopwords') 
from nltk.corpus import stopwords
import nltk
nltk.download()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [ ]:
data = pd.read_csv('datanew.csv',encoding='windows-1254')
data.head()


In [ ]:
# unique ratings 
pd.unique(data['rating'])


In [ ]:
sns.countplot(data=data, 
			x='rating', 
			order=data.rating.value_counts().index)


In [ ]:
# rating label(final) 
pos_neg = [] 
for i in range(len(data['rating'])): 
	if data['rating'][i] >= 5: 
		pos_neg.append(1) 
	else: 
		pos_neg.append(0) 

data['label'] = pos_neg 


In [ ]:
from tqdm import tqdm 


def preprocess_text(text_data): 
	preprocessed_text = [] 

	for sentence in tqdm(text_data): 
		# Removing punctuations 
		sentence = re.sub(r'[^\w\s]', '', sentence) 

		# Converting lowercase and removing stopwords 
		preprocessed_text.append(' '.join(token.lower() 
										for token in nltk.word_tokenize(sentence) 
										if token.lower() not in stopwords.words('english'))) 

	return preprocessed_text 


In [ ]:
preprocessed_review = preprocess_text(data['review'].values) 
data['review'] = preprocessed_review


In [ ]:
data.head()


In [ ]:
data["label"].value_counts()


In [ ]:
consolidated = ' '.join( 
	word for word in data['review'][data['label'] == 1].astype(str)) 
wordCloud = WordCloud(width=1600, height=800, 
					random_state=21, max_font_size=110) 
plt.figure(figsize=(15, 10)) 
plt.imshow(wordCloud.generate(consolidated), interpolation='bilinear') 
plt.axis('off') 
plt.show() 


In [ ]:
cv = TfidfVectorizer(max_features=2500) 
X = cv.fit_transform(data['review'] ).toarray()


In [ ]:
X


In [ ]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, data['label'], 
													test_size=0.33, 
													stratify=data['label'], 
													random_state = 42)


In [ ]:
from sklearn.tree import DecisionTreeClassifier 

model = DecisionTreeClassifier(random_state=0) 
model.fit(X_train,y_train) 

#testing the model 
pred = model.predict(X_train) 
print(accuracy_score(y_train,pred))


In [ ]:
from sklearn import metrics 
cm = confusion_matrix(y_train,pred) 

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = cm, 
											display_labels = [False, True]) 

cm_display.plot() 
plt.show()
